# Variable Definitions demo for creators and owners of Variable definitions
Creator team of is set as variable owner. Only owner can edit a Variable defintion


In [21]:
from datetime import date
from pprint import pprint

from dapla_metadata.variable_definitions.generated.vardef_client.models.contact import (
    Contact,
)
from dapla_metadata.variable_definitions.generated.vardef_client.models.draft import (
    Draft,
)
from dapla_metadata.variable_definitions.generated.vardef_client.models.update_draft import (
    UpdateDraft,
)
from dapla_metadata.variable_definitions.generated.vardef_client.models.validity_period import (
    ValidityPeriod,
)
from dapla_metadata.variable_definitions.generated.vardef_client.models.variable_status import (
    VariableStatus,
)
from dapla_metadata.variable_definitions.vardef import Vardef


## Environment

In [ ]:

from dapla_metadata.variable_definitions.config import get_active_group
from dapla_metadata.variable_definitions.config import get_oidc_token
from dapla_metadata.variable_definitions.config import get_vardef_host

get_active_group()
get_oidc_token()
get_vardef_host()


## Drafts

### Create a draft
Can produce a 409 CONFLICT if the `short_name` already exists.

In [39]:
draft = Draft(
    name = {
        "nb": "test_navn",
        "nn": "test_namn",
        "en": "test_name",
    },
    short_name= "latt",
    definition= {
        "nb": "Skriv en definisjonstekst på norsk bokmål.",
        "nn": "Skriv ein definosjonstekst på nynorsk.",
        "en": "Write a definition text in english.",
    },
    classification_reference="91",
    unit_types=[
        "01",
    ],
    subject_fields=[
        "al",
    ],
    contains_special_categories_of_personal_data=False,
    measurement_type=None,
    valid_from=date(2024,12,18),
    external_reference_uri=None,
    comment=None,
    related_variable_definition_uris=None,
    contact=None,
)

In [ ]:
my_draft = Vardef.create_draft(draft)

In [6]:
invalid_draft = Draft(
    name = {
        "nb": "test_navn",
        "nn": "test_namn",
        "en": "test_name",
    },
    short_name= "lan",
    definition= {
        "nb": "Skriv en definisjonstekst på norsk bokmål.",
        "nn": "Skriv ein definosjonstekst på nynorsk.",
        "en": "Write a definition text in english.",
    },
    classification_reference="91",
    unit_types=[
        "haha",
    ],
    subject_fields=[
        "a",
    ],
    contains_special_categories_of_personal_data=False,
    measurement_type=None,
    valid_from=date(2024,12,18),
    external_reference_uri=None,
    comment=None,
    related_variable_definition_uris=None,
    contact=None,
)

In [ ]:
Vardef.create_draft(invalid_draft)

In [ ]:
# check details
print(my_draft)
# You can either update the variable after each change or use the Vardef.get_variable_definition() with variable id directly

### Edit a draft

This operation must be performed on Variable Definitions with status DRAFT.

In [ ]:
# Test draft exceptions invalid short_name
invalid_short_name = UpdateDraft(
    short_name="_)(45)",
)
my_draft.update_draft(invalid_short_name)



In [ ]:
my_contact = Contact(
    title={
        "nb": "res",
        "nn": "res",
        "en": "res",
        },
    email="sibby@ssb.no",
)
update_contact = UpdateDraft(
    contact=my_contact,
)
my_draft.update_draft(update_draft=update_contact)
print(Vardef.get_variable_definition(my_draft.id))

### Migrate a legacy definition from Vardok to Vardef

Note: This will raise an error if the short name already exists.

In [ ]:
# Migrate from Vardok exception invalid id
Vardef.migrate_from_vardok("21")

In [13]:
# Save variable Vardok
my_vardok_draft = Vardef.migrate_from_vardok("90")
print(my_vardok_draft)

### Delete a draft

This operation must be performed on Variable Definitions with status DRAFT.

In [ ]:
my_vardok_draft.delete_draft()

## Publish
When draft has been quality checked and the resulkt is as desired it is time to Publish the variable, either internal or external. Be aware: this process is irreversibel.

In [ ]:
update_status = UpdateDraft(
    variable_status=VariableStatus.PUBLISHED_INTERNAL,
)

my_draft.update_draft(update_status)

## Patch
### Create a patch
#### Update owner
Owner can be extended or completly transferred to another team.  

#### Update owner by adding owner group

In [ ]:
from dapla_metadata.variable_definitions.generated.vardef_client.models.owner import (
    Owner,
)
from dapla_metadata.variable_definitions.generated.vardef_client.models.patch import (
    Patch,
)

new_owner = Patch(
    owner=Owner(
    team="dapla-felles",
    groups=[
        "dapla-felles-developers",
        "play-enhjoern-a-developers",
    ],
),
)

my_draft.create_patch(new_owner)
my_draft = Vardef.get_variable_definition(my_draft.id)
print(my_draft.owner)

#### Update owner by replacing team and group
be aware of if you remove your team entirely from owner the variable will be forbidden. Feel free to test.But


In [ ]:
replace_owner = Patch(
    owner=Owner(
    team="play-enhjoern-a",
    groups=[
        "play-enhjoern-a-developers",
    ],
),
)

my_draft.create_patch(replace_owner)
my_draft = Vardef.get_variable_definition(my_draft.id)
print(my_draft.owner)
my_draft.create_patch(Patch(unit_types=["01"]))

In [ ]:
#forbidden exception

In [ ]:
# change owner

In [ ]:
# read details
Vardef.get_variable_definition(my_draft.id).subject_fields

In [ ]:
patches = my_draft.list_patches()
pprint(patches)
for patch in patches:
    print(patch)

## Validity period

In [ ]:
# create validity period


invalid_validity_period = ValidityPeriod(
    name={
        "nb": "new name",
    },
    valid_from=date(2025,4,5),
)

my_draft.create_validity_period(invalid_validity_period)

In [ ]:
# validity period exceptions
# correct validity period
# check languages

valid_validity_period = ValidityPeriod(
    definition={
        "nb": "ny definisjon2",
        "nn": "ny definisjon2",
        "en": "new definition2",
    },
    valid_from=date(2040,4,5),
)
my_draft.create_validity_period(valid_validity_period)


In [ ]:
print(Vardef.get_variable_definition(my_draft.id))

In [ ]:
validity_periods = my_draft.list_validity_periods()
for period in validity_periods:
    print(period.valid_from)